# Tailor multiple gates using the integer representation

In  [tutorial.ipynb](./tutorial.ipynb) we learned how to tailor a logical circuit for a stabilizer code. We did this by passing `Connectivity`, `StabilizerCode`, and `Circuit` objects to the `tailor_logical_gate` function. If we want to create circuit implementations for many different logical gates, we may call this function multiple times. If we are looking for a few specific gate implementations, this works fine. On the other hand, one might want to create a large amount of logical gates, say 720 in total; the number of two-qubit Clifford gates modulo Paulis. For this, we can use the function `tailor_multiple_logical_gates`. This function avoids the additional overhead of creating a distinct _Gurobi_ model for all gates, and, more importantly, we can pass a list of integers identifying specific Clifford gates.

In [2]:
import htlogicalgates as htlg

The workflow is similiar to that of `tailor_logical_gate`. The only difference is that, instead of a `Circuit` object, we pass a list of integers.

In [3]:
connectivity = htlg.Connectivity("circular", num_qubits=4)
stab_code = htlg.StabilizerCode("4_2_2")
logical_gates = [0,1,121]
result = htlg.tailor_multiple_logical_gates(stab_code, connectivity, logical_gates, 2)

## Clifford gates represented by integers

For a given number of qubits, every Clifford is identified by an integer. This identification is unique up to Pauli operators and is based on the work _J. Math. Phys. 55, 122202 (2014)_ by Koenig _et. al._ (also available under [arXiv:1406.2170](https://arxiv.org/abs/1406.2170)).

We can work with this representation by using the `Clifford` class. For example, we can turn a `Circuit` object into its corresponding `Clifford` object. Note that all circuits in this package are Clifford circuits.

In [4]:
circ0 = htlg.Circuit("I 1", 2)
circ1 = htlg.Circuit("H 0", 2)
circ2 = htlg.Circuit("H 0\nH 1", 2)
cliff0 = circ0.to_clifford()
cliff1 = circ1.to_clifford()
cliff2 = circ2.to_clifford()
print(f"Clifford integer of cliff0: '{cliff0.clifford_int}'")
print(f"Clifford integer of cliff1: '{cliff1.clifford_int}'")
print(f"Clifford integer of cliff2: '{cliff2.clifford_int}'")

Clifford integer of cliff0: '0'
Clifford integer of cliff1: '1'
Clifford integer of cliff2: '121'


Here, we used the property `clifford_int` to obtain the integer of the Clifford operator modulo Paulis. For completeness, the Pauli part is also identified by an integer.

In [5]:
cliff3 = htlg.Circuit("H 0\nH 1\nX 1").to_clifford()
print(f"Clifford integer of cliff3: '{cliff3.clifford_int}'")
print(f"Pauli integer of cliff3: '{cliff3.pauli_int}'")
print(f"Pauli integer of cliff2: '{cliff2.pauli_int}'")

Clifford integer of cliff3: '121'
Pauli integer of cliff3: '4'
Pauli integer of cliff2: '0'


As shown above, the Clifford integers of both Clifford gates are the same but their Pauli integers differ.

## Using the function `tailor_multiple_gates`

As previously shown, we can use the function `tailor_multiple_gates` in the following manner.

In [6]:
connectivity = htlg.Connectivity("circular", num_qubits=4)
stab_code = htlg.StabilizerCode("4_2_2")
logical_gates = [0, 1, 121]
result = htlg.tailor_multiple_logical_gates(stab_code, connectivity, logical_gates, 2)

The function returns a dictionary containing all tailored gates and meta information.

In [7]:
print(list(result["Meta"].keys()))
print(list(result["Gates"].keys()))

['Connectivity', 'Code', 'n', 'k', 'Number CZ layers', 'Time limit', 'Started', 'Ended']
[0, 1, 121]


For example, we can query the used connectivity and stabilizer code by using `res["Meta"]["Connectivity"]` and `res["Meta"]["Code"]`, respectively. After turning them into `np.array` by using the `eval` function, we can pass them as arguments to the constructor of the respectively classes.

In [8]:
from numpy import array, int32

print(result["Meta"]["Connectivity"])
print(result["Meta"]["Code"])

connectivity = htlg.Connectivity(eval(result["Meta"]["Connectivity"]))
stab_code = htlg.StabilizerCode(eval(result["Meta"]["Code"]))

array([[0, 1, 0, 1],
       [1, 0, 1, 0],
       [0, 1, 0, 1],
       [1, 0, 1, 0]])
array([[1, 1, 0, 0, 1, 0],
       [1, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 1, 1, 0, 1],
       [0, 0, 0, 1, 0, 1],
       [0, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]])


On the other hand, we can query the circuits from the result dictionary by identifying different gates by the integer that represents the logical Clifford action.

In [9]:
result["Gates"][121]

{'Circuit': 'Z 0\nX 0\nX 1\nC_XYZ 1\nX 2\nC_XYZ 2\nCZ 0 3\nCZ 1 2\nC_XYZ 0\nH 1\nH 2\nC_XYZ 3\nCZ 1 2\n',
 'Status': 'Optimal',
 'Runtime': 4.170000076293945}

We can save the dictionary to a .json-file automatically by specifying the `output_file` argument of the function `tailor_multiple_logical_gates`. The runtime is given in seconds and the status message is the same as for the function `tailor_logical_gate`. Note that the circuit is given as a string such that the dictionary is JSON-serializable. Once again, we can turn the string into a `Circuit` object by passing it to the constructor.

In [10]:
circ = htlg.Circuit(result["Gates"][121]["Circuit"])